In [1]:
# Database connection parameters
db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}

In [2]:
import xml.etree.ElementTree as ET
import psycopg2
from datetime import datetime
import re

# XML file path
xml_file_path = 'D:\\W\\RELEX\\RELEX\\Broker and Vendor\\Inbound Files\\VENDOR.xml'

# Open the XML file
with open(xml_file_path, 'r', encoding='utf-8') as file:
    xml_data = file.read()

# Database connection parameters
db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}

# Table name
table_name = "cspoms.XXPO_ORCL2CSPOMS_VEND_STG".upper()

# Split the XML data into lines
lines = xml_data.splitlines()

# Regular expression pattern to match the elements
pattern = re.compile(r"<([^>]+)>([^<]+)</\1>")
tag_to_column = {
    'BICEPSVENDOR': 'BIC_VENDOR_NUMBER',
    'BIC_VENDOR_FACILITY':'BIC_VENDOR_FACILITY',
    'BUYERNUMBER': 'BIC_BUYER_NUMBER',
    'CRPLEVEL': 'BIC_CRP_LEVEL',
    'STATUS': 'BIC_STATUS',
    'PORECOMMENDEDIND': 'BIC_PO_RECOMMENDED_IND',
    'AFETYPE1': 'BIC_AFE_TYPE1',
    'AFEFAXNUMBER1': 'BIC_AFE_FAX_NUMBER',
    'AFECONTACT1': 'BIC_AFE_CONTACT',
    'MINIMUMQUANTITY': 'BIC_MINIMUM_QUANTITY',
    'MINIMUMTYPE': 'BIC_MINIMUM_TYPE',
    'BKTQUANTITY1': 'BIC_BRACKET_QUANTITY',
    'CURRENTBKTNUMBER': 'BIC_CURRENT_BKT_NUMBER',
    'CURRENTBKTQUANTITY': 'BIC_CURRENT_BKT_QUANTITY',
    'CURRENTBKTTYPE': 'BIC_CURRENT_BKT_TYPE',
    'MAXIMUMQUANTITY': 'BIC_MAXIMUM_QUANTITY',
    'MAXIMUMTYPE': 'BIC_MAXIMUM_TYPE',
    'ORDERINTERVALWEEKS': 'BIC_ORDER_INTERVAL_WEEKS',
    'LEADTIMESTATEDWEEKS': 'BIC_LEAD_TIME_STATED_WEEKS',
    'FIXEDREVIEWDAY1': 'BIC_FIXED_REVIEW_DAY1',
    'FIXEDREVIEWDAY2': 'BIC_FIXED_REVIEW_DAY2',
    'FIXEDREVIEWDAY3': 'BIC_FIXED_REVIEW_DAY3',
    'FIXEDREVIEWDAY4': 'BIC_FIXED_REVIEW_DAY4',
    'FIXEDREVIEWDAY5': 'BIC_FIXED_REVIEW_DAY5',
    'FIXEDREVIEWDAY6': 'BIC_FIXED_REVIEW_DAY6',
    'FIXEDREVIEWDAY7': 'BIC_FIXED_REVIEW_DAY7',
    'TARGETSERVICELEVEL': 'BIC_TARGET_SERVICE_LEVEL',
    'NAME': 'BIC_NAME',
    'ADDRESS1': 'BIC_ADDRESS_1',
    'CITY': 'BIC_CITY',
    'STATE': 'BIC_STATE',
    'ZIP': 'BIC_ZIP',
    'PHONE': 'BIC_PHONE',
    'CONTACT': 'BIC_CONTACT',
    'TRANSFERMATCHTYPE': 'BIC_TRANSFER_MATCH_TYPE',
    'PAYABLEVENDORNBR': 'BIC_PAYABLE_VENDOR_NBR',
    'LEADTIMEWEEKSTYPE': 'BIC_LEAD_TIME_WEEKS_TYPE',
    'DUNSREMITNO': 'BIC_DUNS_REMIT_NO',
    'DUNSREMITSUFFIX': 'BIC_DUNS_REMIT_SUFFIX',
    'FACILITYSHIPTO': 'BIC_FACILITY_SHIP_TO',
    'PRESCHED': 'BIC_PRE_SCHED_FLAG',
    'BROKERNUMBER': 'BIC_BROKER_NUMBER',
    'FREIGHTALLOW': 'BIC_FREIGHT_ALLOW',
    'FREIGHTALLOWTYPE': 'BIC_FREIGHT_ALLOW_TYPE',
    'TERMSPERCENT': 'BIC_TERMS_PERCENT',
    'TERMSDAYS': 'BIC_TERMS_DAYS',
    'TERMSNETDAYS': 'BIC_TERMS_NET_DAYS',
    'TERMSBASE': 'BIC_TERMS_BASE',
    'BILLFORSHORTEDDEALSFLAG': 'BIC_BILL_FOR_SHORT_DEALS_FLAG',
    'TRANSITDAYS': 'BIC_TRANSIT_DAYS',
    'DUNSNO': 'BIC_DUNS_NO',
    '2NDMINQUANTITY': 'BIC_2ND_MIN_QUANTITY',
    '2NDMINTYPE': 'BIC_2ND_MIN_TYPE',
    '2NDMAXQUANTITY': 'BIC_2ND_MAX_QUANTITY',
    '2NDMAXTYPE': 'BIC_2ND_MAX_TYPE',
    'FLAGPREPAID': 'BIC_FLAG_PREPAID',
    'FLAGPREPAYANDADD': 'BIC_FLAG_PREPAY_AND_ADD',
    'FLAGFREIGHTBILL': 'BIC_FLAG_FREIGHT_BILL',
    'FLAGBACKHAUL': 'BIC_FLAG_BACKHAUL',
    'FLAGTRUCK': 'BIC_FLAG_TRUCK',
    'FLAGRAIL': 'BIC_FLAG_RAIL',
    'BACKHAUL': 'BIC_BACKHAUL',
    'BACKHAULTYPE': 'BIC_BACKHAUL_TYPE',
    'FREIGHTBILL': 'BIC_FREIGHTBILL',
    'ORDERSEQUENCE': 'BIC_ORDER_SEQUENCE',
    'SSAALLOWPERCENT': 'BIC_SSA_ALLOW_PERCENT',
    'SSAALLOWBASIS': 'BIC_SSA_ALLOW_BASIS',
    'SSAALLOWACCOUNT': 'BIC_SSA_ALLOW_ACCOUNT',
    'TRANSACTIONID': 'BIC_TRANSACTION_ID',
    'COUNTRY': 'BIC_COUNTRY',
    'ORDERFILTER': 'BIC_FILTER_VENDOR_FLAG',
    'LBLIMIT': 'BIC_LOAD_BLDG_LIMIT_FLAG',
    'WAREHOUSECODE': 'BIC_WAREHOUSE_CODE',
    'FLAGEXTBCKHL': 'BIC_EXTERNAL_BACKHAUL'
}
print('tag_to_column : ',len(tag_to_column))
# Loop through each line in the XML data
for line in lines:
    # Skip empty lines or lines containing only whitespace
    if not line.strip():
        continue

    # Define a dictionary to store values
    element_values = {}

    # Find all matches in the text data
    matches = re.findall(pattern, line)

    # Store the matches in the dictionary
    for tag, value in matches:
        element_values[tag] = value.strip()
    # Assuming 'BICEPSVENDOR' is present in element_values
    if 'BICEPSVENDOR' in element_values:
        bicepsvendor_value = element_values['BICEPSVENDOR']

        # Check if the value has at least 3 characters
        if len(bicepsvendor_value) >= 3:
            # Extract the first 3 digits
            facility_value = bicepsvendor_value[:3]

            # Assign the first 3 digits to a new key
            element_values['BIC_VENDOR_FACILITY'] = facility_value

            # Update 'BICEPSVENDOR' with the remaining value
            element_values['BICEPSVENDOR'] = bicepsvendor_value[3:]
    print('element_values:############',element_values)
    # Now 'BIC_VENDOR_FACILITY' and updated 'BICEPSVENDOR' are present in element_values
    # Replace column names with the ones from tag_to_column
    mapped_values = {tag_to_column[key]: value for key, value in element_values.items() if key in tag_to_column}
    print('mapped_values:########0',mapped_values)
    # Generate SQL INSERT statement
    columns_str = ",".join(mapped_values.keys())
    values_str = ",".join(f"'{v}'" for v in mapped_values.values())

    insert_query = f"INSERT INTO {table_name} ({columns_str}) VALUES ({values_str});"
    print(insert_query)

    # Establish a database connection and execute the INSERT statement
#     with psycopg2.connect(**db_params) as conn:
#         with conn.cursor() as cursor:
#             cursor.execute(insert_query)

    print(f"Record inserted: {mapped_values}")


tag_to_column :  74
element_values:############ {'BICEPSVENDOR': '445932', 'BUYERNUMBER': 'J8', 'CRPLEVEL': '', 'STATUS': 'A', 'PORECOMMENDEDIND': 'B', 'AFETYPE1': '', 'AFEFAXNUMBER1': '', 'AFECONTACT1': '', 'MINIMUMQUANTITY': '0000008', 'MINIMUMTYPE': 'P', 'BKTQUANTITY1': '0000001', 'CURRENTBKTNUMBER': '0', 'CURRENTBKTQUANTITY': '0000026', 'CURRENTBKTTYPE': 'P', 'MAXIMUMQUANTITY': '0000026', 'MAXIMUMTYPE': 'P', 'ORDERINTERVALWEEKS': '000.00', 'LEADTIMESTATEDWEEKS': '002.01', 'FIXEDREVIEWDAY1': '', 'FIXEDREVIEWDAY2': '', 'FIXEDREVIEWDAY3': '', 'FIXEDREVIEWDAY4': '', 'FIXEDREVIEWDAY5': '', 'FIXEDREVIEWDAY6': '', 'FIXEDREVIEWDAY7': '', 'TARGETSERVICELEVEL': '0095', 'NAME': 'INTL. TRADE IMPORTS', 'ADDRESS1': '253 CASTLEBERRY INDUSTRIAL DR', 'CITY': 'CUMMINGS', 'STATE': 'GA', 'ZIP': '30040', 'PHONE': '404 795 0352', 'CONTACT': 'RICARDO SANCHEZ', 'TRANSFERMATCHTYPE': 'I', 'PAYABLEVENDORNBR': '73161', 'LEADTIMEWEEKSTYPE': 'B', 'DUNSREMITNO': '000000000', 'DUNSREMITSUFFIX': '', 'FACILITYSHIPT

In [7]:

element_values

{'BICEPSVENDOR': '008571',
 'BUYERNUMBER': 'ZZ',
 'CRPLEVEL': '',
 'STATUS': 'A',
 'PORECOMMENDEDIND': 'B',
 'AFETYPE1': '',
 'AFEFAXNUMBER1': '',
 'AFECONTACT1': '',
 'MINIMUMQUANTITY': '0000001',
 'MINIMUMTYPE': 'U',
 'BKTQUANTITY1': '0000001',
 'CURRENTBKTNUMBER': '0',
 'CURRENTBKTQUANTITY': '0000001',
 'CURRENTBKTTYPE': 'U',
 'MAXIMUMQUANTITY': '0042000',
 'MAXIMUMTYPE': 'L',
 'ORDERINTERVALWEEKS': '001.00',
 'LEADTIMESTATEDWEEKS': '001.00',
 'FIXEDREVIEWDAY1': '',
 'FIXEDREVIEWDAY2': '',
 'FIXEDREVIEWDAY3': '',
 'FIXEDREVIEWDAY4': '',
 'FIXEDREVIEWDAY5': '',
 'FIXEDREVIEWDAY6': '',
 'FIXEDREVIEWDAY7': '',
 'TARGETSERVICELEVEL': '0095',
 'NAME': 'MCELRONE SALES INC',
 'ADDRESS1': '445 GODWIN AVENUE',
 'CITY': 'MIDLAND PARK',
 'STATE': 'NJ',
 'ZIP': '07432',
 'PHONE': '201 262 9900',
 'CONTACT': 'NANCY MCELRONE',
 'TRANSFERMATCHTYPE': 'I',
 'PAYABLEVENDORNBR': '88888',
 'LEADTIMEWEEKSTYPE': 'C',
 'DUNSREMITNO': '000000000',
 'DUNSREMITSUFFIX': '',
 'FACILITYSHIPTO': '00',
 'PRESCHED

In [1]:

# Establish a database connection and execute the INSERT statement
with psycopg2.connect(**db_params) as conn:
    with conn.cursor() as cursor:
    # Execute the SQL statement
        cursor.execute("SELECT nextval('cs_vendor_id_seq')")
        # Fetch the result into a variable
        new_cs_vendor_id = cursor.fetchone()[0]
        # Execute the SQL statement to select all rows from the table
        cursor.execute("SELECT * FROM XXPO_ORCL2CSPOMS_VEND_STG")

        # Fetch all rows into a list of tuples
        result_rows = cursor.fetchall()

        # Get the column names from the cursor description
        column_names = [desc[0] for desc in cursor.description]

        # Create a DataFrame using the result and column names
        stg_df = pd.DataFrame(result_rows, columns=column_names)

        # Print or use the DataFrame as needed
        print(stg_df)
        # Extract the 'bic_vendor_number' column and convert it to a list
        bic_vendor_numbers_list = stg_df['bic_vendor_number'].tolist()

        # Convert the list of strings to a list of integers
        bic_vendor_numbers_list = list(map(int, bic_vendor_numbers_list))

        # Print or use the list as needed
        print(bic_vendor_numbers_list)


# Print or use the new value as needed
print("New CS_VENDOR_ID:", new_cs_vendor_id)

NameError: name 'psycopg2' is not defined

In [41]:
import pandas as pd
import cx_Oracle

# Replace these values with your Oracle database connection details
username = 'apps'
password = 'apps'
host = 'csebsd2db.cswg.com' #EBSD2
port = '1521'
service_name = 'csebsd2_int' #EBSD2

# Create a connection
dsn = cx_Oracle.makedsn(host, port, service_name=service_name)
connection = cx_Oracle.connect(username, password, dsn)

# Convert the list of numbers to a string for the SQL query
# vendor_numbers_str = ', '.join(map(str, bic_vendor_numbers_list))
# Convert the list of numbers to a string with quotes for the SQL query
vendor_numbers_str = ', '.join([f"'{num}'" for num in bic_vendor_numbers_list])


# Construct the SQL query
sql_query = f"SELECT * FROM APPS.XXAP_VENDOR_MASTER WHERE VENDOR_NBR IN ({vendor_numbers_str})"

oracle_df = pd.read_sql(sql_query, connection)
oracle_df['VENDOR_NBR'] = oracle_df['VENDOR_NBR'].str.zfill(6)
# Display the DataFrame
oracle_df.head()


,VENDOR_SITE_ID,VENDOR_NBR,VENDOR_NAME,VENDOR_START_DATE,VENDOR_END_DATE,VENDOR_STATUS,VENDOR_TYPE,AP_VENDOR_ID,AP_VENDOR_NAME,AP_VENDOR_ALT_NAME,...,PICKUP_DAY_6,PICKUP_DAY_7,EOQ_CAP_DAYS,WHSE_SHIP_TO,SHIP_WITH_VENDOR,RANK_PERCENT_OVERRIDE,PO_DATE_CHANGE_RSN_FLAG,PULLBKS_AVAILABLE,HAWAII_TRANS_METHOD,TEST_NEW
0,3275707,445932,None,2022-07-20 09:38:30,None,A,Buyer Vendor,1935048,INTERNATIONAL TRADE IMPORTS LLC,None,...,None,None,None,None,None,None,None,None,None,None
1,3276057,447709,None,2022-08-05 08:17:43,None,A,Buyer Vendor,1935048,INTERNATIONAL TRADE IMPORTS LLC,None,...,None,None,None,None,None,None,None,None,None,None
2,3218150,008571,McELRONE SALES INC,2022-02-25 11:02:50,None,A,Broker,992011,C&S Dummy Broker AP,None,...,None,None,None,None,None,None,None,None,None,None


In [35]:
259 + 121

380

In [30]:
bic_vendor_numbers_list
bic_vendor_numbers_list = [8571, 8571, 445932, 447709]

# Pad the left side of each number to 6 digits
padded_bic_vendor_numbers = [str(num).zfill(6) for num in bic_vendor_numbers_list]

print(padded_bic_vendor_numbers)


['008571', '008571', '445932', '447709']


In [58]:

# Merge oracle_df with stg_df based on the common column 'bic_vendor_number' and 'VENDOR_NBR'
merged_df = pd.merge(stg_df, oracle_df, how='left', left_on='bic_vendor_number', right_on='VENDOR_NBR')

# Display the merged DataFrame
print(merged_df)
for i in merged_df.columns:
    
    print(i)

   stg_seq_id bic_vendor_facility bic_vendor_number bic_buyer_number  \
0          11                 252            008571               ZZ   
1          12                 905            008571               ZZ   
2          13                 251            445932               J8   
3          14                 252            447709               J8   

  bic_crp_level bic_status bic_po_recommended_ind bic_afe_type1  \
0                        A                      B                 
1                        A                      B                 
2                        A                      B                 
3                        A                      B                 

  bic_afe_fax_number bic_afe_contact  ... PICKUP_DAY_6 PICKUP_DAY_7  \
0                                     ...         None         None   
1                                     ...         None         None   
2                                     ...         None         None   
3                  

In [79]:
XVM_df = merged_df[["VENDOR_SITE_ID",'VENDOR_NBR',
"BUYER_VENDOR",'CONNECTING_VENDOR','BUYER_VENDOR_NAME','bic_vendor_facility','SHIP_TO_LOCATION_NAME','VENDOR_STATUS'
,'AP_VENDOR_NBR','AP_VENDOR_NAME','BICEPS_BUYER_CODE','BUYER_NAME'

                    ,'bic_warehouse_code'
,'VENDOR_FOODLINK_FLG'
,'ADDR_LINE1'
,'ADDR_LINE2'
,'ADDR_LINE3'
,'CITY'
,'STATE'
,'ZIP_CODE'
,'COUNTRY'
,'PHONE_NBR'
,'VENDOR_CONTACT_NAME'
,'SHIP_TO_LOCATION_NAME'
,'BILL_TO_LOCATION_NAME'
                    
,'VENDOR_BROKER_NBR'
,'VENDOR_BROKER_NAME'
                   
,'bic_flag_truck'
,'bic_flag_rail'
,'bic_afe_contact'
,'bic_afe_fax_number']]
# ,'BROKER ADDRESS_1'
# ,'BROKER ADDRESS_2'
# ,'BROKER CITY'
# ,'BROKER STATE'
# ,'BROKER ZIP'
# ,'BROKER COUNTRY'




for i in XVM_df.columns:
    print(i)

VENDOR_SITE_ID
VENDOR_NBR
BUYER_VENDOR
CONNECTING_VENDOR
BUYER_VENDOR_NAME
bic_vendor_facility
SHIP_TO_LOCATION_NAME
VENDOR_STATUS
AP_VENDOR_NBR
AP_VENDOR_NAME
BICEPS_BUYER_CODE
BUYER_NAME
bic_warehouse_code
VENDOR_FOODLINK_FLG
ADDR_LINE1
ADDR_LINE2
ADDR_LINE3
CITY
STATE
ZIP_CODE
COUNTRY
PHONE_NBR
VENDOR_CONTACT_NAME
SHIP_TO_LOCATION_NAME
BILL_TO_LOCATION_NAME
VENDOR_BROKER_NBR
VENDOR_BROKER_NAME
bic_flag_truck
bic_flag_rail
bic_afe_contact
bic_afe_fax_number


In [80]:
import pandas as pd


# Generate the INSERT statement with ON CONFLICT
insert_statement = f"INSERT INTO cspoms.XXPO_VENDOR_MASTER ({', '.join(XVM_df.columns)}) VALUES\n"

for index, row in XVM_df.iterrows():
    values = ', '.join([repr(val) for val in row.values])
    insert_statement += f"({values}),\n"

# Remove the trailing comma and newline
insert_statement = insert_statement.rstrip(',\n')

# Add ON CONFLICT clause for upsert
conflict_columns = ['VENDOR_NUMBER', 'VENDOR_FACILITY']  # Specify the conflict columns
update_set_clause = ', '.join([f"{col} = EXCLUDED.{col}" for col in XVM_df.columns])
insert_statement += f"\nON CONFLICT ({', '.join(conflict_columns)}) DO UPDATE SET {update_set_clause};"

print(insert_statement)


INSERT INTO cspoms.XXPO_VENDOR_MASTER (VENDOR_SITE_ID, VENDOR_NBR, BUYER_VENDOR, CONNECTING_VENDOR, BUYER_VENDOR_NAME, bic_vendor_facility, SHIP_TO_LOCATION_NAME, VENDOR_STATUS, AP_VENDOR_NBR, AP_VENDOR_NAME, BICEPS_BUYER_CODE, BUYER_NAME, bic_warehouse_code, VENDOR_FOODLINK_FLG, ADDR_LINE1, ADDR_LINE2, ADDR_LINE3, CITY, STATE, ZIP_CODE, COUNTRY, PHONE_NBR, VENDOR_CONTACT_NAME, SHIP_TO_LOCATION_NAME, BILL_TO_LOCATION_NAME, VENDOR_BROKER_NBR, VENDOR_BROKER_NAME, bic_flag_truck, bic_flag_rail, bic_afe_contact, bic_afe_fax_number) VALUES
(3218150, '008571', None, None, None, '252', 'NFR - C&S - Optical Ave', 'A', '63918', 'C&S Dummy Broker AP', None, None, None, None, '445 GODWIN AVENUE', None, None, 'MIDLAND PARK', 'NJ', '07432', 'US', None, 'NANCY MCELRONE', 'NFR - C&S - Optical Ave', 'NFR - Brattleboro, VT', None, None, 'Y', '', '', ''),
(3218150, '008571', None, None, None, '905', 'NFR - C&S - Optical Ave', 'A', '63918', 'C&S Dummy Broker AP', None, None, None, None, '445 GODWIN AVENU

In [ ]:
INSERT INTO cspoms.XXPO_VENDOR_MASTER ( ) values ( );